In [1]:
 !pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"l4e1owek","key":"a31d8529a9ceb19f66a1f35ce6044f83"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c ml-intensive-yandex-academy-spring-2024 -p data

 99% 976M/988M [00:15<00:00, 97.9MB/s]
100% 988M/988M [00:15<00:00, 67.3MB/s]


In [6]:
import zipfile
import os

# Путь к вашему архиву
zip_file_path = '/content/data/ml-intensive-yandex-academy-spring-2024.zip'

# Папка, куда вы хотите извлечь содержимое архива
extract_folder = '/content/'

# Создаем папку для извлеченных файлов, если она еще не существует
os.makedirs(extract_folder, exist_ok=True)

# Разархивируем архив
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Архив успешно разархивирован.")

Архив успешно разархивирован.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
import torch.nn as nn
import torch


class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        return self.block(x)

# Определение класса блока декодера
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.block(x)


class LungMaskGeneratorV4(nn.Module):
    def __init__(self):
        super(LungMaskGeneratorV4, self).__init__()
        self.encoder = nn.Sequential(
            EncoderBlock(3, 64),
            EncoderBlock(64, 128),
            EncoderBlock(128, 256),
            EncoderBlock(256, 512),
            EncoderBlock(512, 1024)
        )
        self.decoder = nn.Sequential(
            DecoderBlock(1024, 512),
            DecoderBlock(512, 256),
            DecoderBlock(256, 128),
            DecoderBlock(128, 64),
            DecoderBlock(64, 32),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [10]:
vae_model = LungMaskGeneratorV4().to(device)
c3 = torch.load('/content/drive/MyDrive/lung_mask_generatorV414.pth', map_location=device)
vae_model.load_state_dict(c3)

<All keys matched successfully>

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Изменение количества входных каналов на 3
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 3)  # Вывод: 3 класса

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [12]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import random as rnd
import pandas as pd
import numpy as np
import pickle
import torch
import cv2
import os


def resize_image(image, size):
    original_image = Image
    width, height = original_image.size
    aspect_ratio = width / height

    # Выбираем масштаб по меньшей стороне
    if width > height:
        new_width = size
        new_height = int(size / aspect_ratio)
    else:
        new_width = int(size * aspect_ratio)
        new_height = size

    resized_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
    return resized_image


class CustomDataset(Dataset):
    def __init__(
        self,
        csv_file,
        root_dir,
        transform=None,
        vae_model=None,
        have_mask=False,
        mask_transform=None
    ):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.have_mask = have_mask
        self.vae_model = vae_model
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, f"img_{self.data_frame.iloc[idx, 0]}.png")
        image = resize_image(Image.open(img_name).convert('RGB'), 224)
        target = self.data_frame.iloc[idx, 1]

        if self.have_mask:
            self.vae_model.eval()
            with torch.no_grad():
                # Применяем модель vae_model для получения маски
                mask = self.vae_model((self.mask_transform(image).unsqueeze(0)).to(device))
                # Преобразуем маску обратно в изображение PIL
                mask = transforms.ToPILImage()(mask.squeeze(0))
                image = Image.fromarray(cv2.bitwise_and(np.array(image), np.array(image), mask=np.array(mask)))

        if self.transform:
            image = self.transform(image)

        return image, target


In [13]:
mean = [0.51394627, 0.51394627, 0.51394627]
std = [0.24807256, 0.24807256, 0.24807256]

transform_for_CNN = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(45),  # Случайный поворот на ±45 градусов
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Изменение цвета
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_for_vae = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [14]:
train_image_dir = '/content/data/train_images'
test_image_dir = '/content/data/test_images'
csv_file = '/content/data/train_answers.csv'

In [15]:
from torch.utils.data import Dataset
import glob
from PIL import Image
import random as rnd
import pandas as pd
import numpy as np
import pickle
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
from torch.utils.data import random_split

In [16]:
train_dataset = CustomDataset(csv_file=csv_file,
                         root_dir=train_image_dir,
                           transform=transform_for_CNN,
                             mask_transform=transform_for_vae,
                               have_mask=True,
                               vae_model=vae_model)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

In [17]:
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [18]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
def train_model(model, criterion, optimizer, train_loader, test_loader, epochs, device):
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_idx, (data, target) in loop:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        loop.set_description(f"Epoch [{epochs[0] + 1}/{epochs[1]}]")
        loop.set_postfix(loss=loss.item())


In [20]:
from sklearn.metrics import f1_score

def calculate_f1_score(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    f1_score_value = f1_score(all_targets, all_predictions, average='weighted')
    return f1_score_value

In [22]:
checkpoint_dir = '/content/drive/MyDrive/'

losses = []
# Load checkpoint if it exists
try:
  checkpoint_path = max(glob.glob(os.path.join(checkpoint_dir, 'checkpoint_CNN_model_epoch_*.pth')), key=os.path.getctime)
except:
  checkpoint_path = ''
print(checkpoint_path)

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location = device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    losses = checkpoint['losses']
else:
    start_epoch = 0

# Training loop with checkpointing
num_epochs = 0
for epoch in range(start_epoch, start_epoch + (num_epochs - start_epoch)):
    model.to(device)
    model.train()
    train_model(model, criterion, optimizer, train_loader, test_loader, [epoch, num_epochs], device)
    f1_score_value = calculate_f1_score(model, test_loader, device)
    print(f"F1-score on test set: {f1_score_value}")

    print("Training completed.")

    checkpoint_path = os.path.join('/content/drive/MyDrive', f'checkpoint_CNN_model_epoch_{epoch + 1}.pth')
    if not os.path.exists(checkpoint_path):
        with open(checkpoint_path, "w") as f:
            print(f'created{checkpoint_path}')
    torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'losses': losses
            }, checkpoint_path)

/content/drive/MyDrive/checkpoint_CNN_model_epoch_36.pth


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
vae_model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

predictions = []

for filename in os.listdir(test_image_dir):
    img_path = os.path.join(test_image_dir, filename)
    img = Image.open(img_path).convert('RGB').resize((224, 224))

    with torch.no_grad():
        mask = vae_model((transform_for_vae(img).unsqueeze(0)).to(device))
        mask = transforms.ToPILImage()(mask.squeeze(0))
        img = Image.fromarray(cv2.bitwise_and(np.array(img), np.array(img), mask=np.array(mask)))

        img = transform(img).unsqueeze(0).to(device)

        output = model(img)
        _, predicted_class = torch.max(output, 1)
        predictions.append((int(filename.split('.')[0].split('_')[1]), predicted_class.item()))

submission_df = pd.DataFrame(predictions, columns=['id', 'target_feature'])
submission_df.to_csv('submission.csv', index=False)